In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
import cesium
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from cesium import datasets
from cesium import featurize as ft

import scipy
from scipy.stats import pearsonr, spearmanr
from scipy.stats import skew

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
print(cesium.__version__)
print(xgb.__version__)
print(scipy.__version__)
print(sklearn.__version__)

## Load data and generate some features of interest

In [ ]:
eeg = datasets.fetch_andrzejak()

In [ ]:
type(eeg)

In [ ]:
eeg.keys()

### Visually inspect

In [ ]:
plt.subplot(3, 1, 1)
plt.plot(eeg["measurements"][0])
plt.legend(eeg['classes'][0])
plt.subplot(3, 1, 2)
plt.plot(eeg["measurements"][300])
plt.legend(eeg['classes'][300])
plt.subplot(3, 1, 3)
plt.plot(eeg["measurements"][450])
plt.legend(eeg['classes'][450])

In [ ]:
type(eeg["measurements"][0])

In [ ]:
type(eeg)

In [ ]:
eeg.keys()

In [ ]:
type(eeg['measurements'])

In [ ]:
len(eeg['measurements'])

In [ ]:
eeg['measurements'][0].shape

## Generate the features

In [ ]:
# from cesium import featurize as ft
# features_to_use = ["amplitude",
#                    "percent_beyond_1_std",
#                    "percent_close_to_median",
#                   "skew",
#                   "max_slope"]
# fset_cesium = ft.featurize_time_series(times=eeg["times"],
#                                               values=eeg["measurements"],
#                                               errors=None,
#                                               features_to_use=features_to_use,
#                                              scheduler = None)

In [ ]:
fset_cesium = pd.read_csv("data/full_eeg_data_features.csv", header = [0, 1])

In [ ]:
fset_cesium.head()

In [ ]:
# fset_cesium.to_csv("full_eeg_data_features.csv")

In [ ]:
fset_cesium.shape

## Exercise: validate/calculate these features by hand
#### look up feature definitions here: http://cesium-ml.org/docs/feature_table.html
confirm the values by hand coding these features for the first EEG measurement
(that is eeg["measurements"][0])

In [ ]:
ex = eeg["measurements"][0]
ex_mean = np.mean(ex)
ex_std  = np.std(ex)

## Prepare data for training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
     fset_cesium.iloc[:, 1:6].values, eeg["classes"], random_state=21)

## Try a random forest with these features

In [ ]:
clf = RandomForestClassifier(n_estimators=10, max_depth=3,
                              random_state=21)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
np.unique(y_test, return_counts=True)

In [ ]:
y_test

In [ ]:
y_test.shape

In [ ]:
y_train.shape

## Try XGBoost with these features

In [ ]:
model = xgb.XGBClassifier(n_estimators=10, max_depth=3,
                              random_state=21)
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model.score(X_train, y_train)

In [ ]:
xgb.plot_importance(model)

## Time Series Forecasting with Decision Trees

### Explore the data

In [ ]:
ap = pd.read_csv("data/AirPassengers.csv", parse_dates=[0])

In [ ]:
ap.head()

In [ ]:
ap.set_index('Month', inplace=True)

In [ ]:
ap.head()

In [ ]:
plt.plot(ap)

In [ ]:
plt.plot(np.diff(np.log(ap.values[:, 0])))

In [ ]:
ts = np.diff(np.log(ap.values[:, 0]))

## Exercise: now that we have 1 time series, how can we convert it to many samples?

In [5]:
NSTEPS = 12

## Exercise: now that we have the time series broken down into a set of samples, how to featurize?

In [ ]:
measures = [vals[i][0:(NSTEPS - 1)] for i in range(vals.shape[0])]

In [ ]:
times = [[j for j in range(NSTEPS - 1)] for i in range(vals.shape[0])]

## Exercise: can you fit an XGBRegressor to this problem? Let's use the first 100 'time series' as the training data

### RMSE can be hard to digest .... Use other assessments to determine how well the model performs

### What went wrong? Let's revisit the feature set

In [ ]:
fset_ap.head()

In [ ]:
plt.plot(vals[0])
plt.plot(vals[1])
plt.plot(vals[2])

## We need to find a way to generate features that encode positional information

### now we will generate our own features

In [ ]:
vals.shape

In [ ]:
feats = np.zeros( (vals.shape[0], 6), dtype = np.float32)
for i in range(vals.shape[0]):
    feats[i, 0] = np.where(vals[i] == np.max(vals[i]))[0][0]
    feats[i, 1] = np.where(vals[i] == np.min(vals[i]))[0][0]
    feats[i, 2] = feats[i, 0] - feats[i, 1]
    feats[i, 3] = np.max(vals[i][-3:])
    feats[i, 4] = vals[i][-1] - vals[i][-2]
    feats[i, 5] = vals[i][-1] - vals[i][-3]

In [ ]:
feats[0:3]

### How do these look compared to the first set of features?

In [ ]:
pd.DataFrame(feats[0:3])

In [ ]:
X_train, y_train = feats[:100, :], outcomes[:100]
X_test, y_test   = feats[100:, :], outcomes[100:]

In [ ]:
model = xgb.XGBRegressor(n_estimators=20, max_depth=2,
                              random_state=21)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=True)

In [ ]:
plt.scatter(model.predict(X_test), y_test)

In [ ]:
print(pearsonr(model.predict(X_test), y_test))
print(spearmanr(model.predict(X_test), y_test))

In [ ]:
plt.scatter(model.predict(X_train), y_train)

In [ ]:
print(pearsonr(model.predict(X_train), y_train))
print(spearmanr(model.predict(X_train), y_train))